<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP_2022/blob/main/03%20Machine%20Learning/notebooks/14-Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clustering</h1>

* El análisis de agrupamiento, o agrupamiento, es una tarea de aprendizaje automático no supervisada.

* Implica descubrir automáticamente la agrupación natural de los datos. A diferencia del aprendizaje supervisado (como el modelado predictivo), los algoritmos de agrupación solo interpretan los datos de entrada y encuentran grupos o agrupaciones naturales en el espacio de características.

* Un grupo, o cluster, es un en el espacio de características donde las instancias están más cerca del grupo que de otros grupos.

* Es probable que estos grupos reflejen algún mecanismo en funcionamiento en el dominio del que se extraen las instancias, un mecanismo que hace que algunas instancias tengan un parecido más fuerte entre sí que con las instancias restantes.

* La agrupación en clústers puede ser útil como actividad de análisis de datos para obtener más información sobre el dominio del problema, el llamado descubrimiento de patrones o descubrimiento de conocimiento.

* El agrupamiento también puede ser útil como un tipo de ingeniería de características, donde los ejemplos existentes y nuevos se pueden mapear y etiquetar como pertenecientes a uno de los grupos identificados en los datos.

* La evaluación de los grupos identificados es subjetiva y puede requerir un experto en el dominio, aunque existen muchas medidas cuantitativas específicas de los grupos.

&#128214; <u>Referencias bibliográficas</u>:
* Flach, Peter (2012). Machine Learning: The Art and Science of Algorithms that Make Sense of Data. Cambridge University Press.

[Algoritmos de clustering en scikit-learn](https://scikit-learn.org/stable/modules/clustering.html)

___

# Demostración ilustrativa de algunos algoritmos de agrupamiento

In [ ]:
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, OPTICS, AgglomerativeClustering

Generación de datos sintéticos

In [ ]:
import numpy as np
from sklearn.datasets import make_classification, make_blobs
import matplotlib.pyplot as plt

# X, y = make_blobs(n_samples=500,centers=3, random_state=24)
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=4)

plt.figure()
plt.scatter(X[:, 0], X[:, 1],c=y)
plt.xticks([])
plt.yticks([])
plt.show()

A diferencia del aprendizaje supervisado, en los métodos de clustering (implementados en scikit-learn) el proceso es de la siguiente manera:

1. Inicializar el objeto, por ejemplo `modelo = KMeans(n_clusters=3)`.
2. Hacer fit: `modelo.fit(X)`. Hay dos opciones:
    * Obtener la lista de etiquetas de clusters como `y_clusters = modelo.predict(X)`.
    * Obtener la lista de etiquetas de clusters usando el atributo `labels_` como `y_clusters = modelo.labels_`.

También pueden obtenerse las etiquetas de los clusters directamente con el método `fit_transform()`.

En esta notebook estaremos usando indistintamente los 3 métodos para ejemplificar su uso.

## [K-MEANS](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Probemos varios valores de `n_clusters`





In [ ]:
from sklearn.cluster import KMeans

modelo = KMeans(n_clusters=3)

modelo.fit(X)
y_clusters = modelo.predict(X)

clusters = np.unique(y_clusters)

for cluster in clusters:
    fila = np.where(y_clusters == cluster)
    plt.scatter(X[fila, 0], X[fila, 1])

plt.show()

Veamos los centroides de cada cluster

In [ ]:
centers = modelo.cluster_centers_

plt.scatter(X[:, 0], X[:, 1],c=y_clusters)
plt.scatter(centers[:,0],centers[:,1],color='black', marker='x',s=80)

plt.show()

### ¿Cómo sabemos cuántos clusters buscar?

Exploremos el **elbow value**.

Para esto, hay que graficar los valores de inercia de cada módelo de K-means para varios valores de $k$. El valor *elbow* representa el número óptimo de clusters, queda determinado por el *codo* en la curva graficada.

In [ ]:
max_num_clusters = 20

inertias = []
k_values = list(range(1,max_num_clusters))
for k in k_values:
    modelo = KMeans(n_clusters=k, n_init='auto')
    modelo.fit(X)
    inertias.append(modelo.inertia_)

plt.figure(figsize=(7,5))
plt.plot(k_values,inertias,color='red')
plt.axvline(x=3,linestyle='dashed',color='gray')
plt.ylabel("Inertia", fontsize=15)
plt.xlabel("Value of k", fontsize=15)
plt.xticks(k_values)
plt.show()

**Silhoutte Score**

También podemos usar el **score de silueta**, el cual es un valor $-1\leq s\leq 1$ que mide que tan coherente son los puntos dentro de sus propios clusters, en términos de las distancias a los demás clusters. Entre más alto el valor, la configuración del clúster es apropiada. Este score es intrínseco del clustering.

Usaremos la implementación de [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).

In [ ]:
from sklearn.metrics import silhouette_score

max_num_clusters = 20

siluetas = []
k_values = list(range(2,max_num_clusters))
for k in k_values:
    kmeans = KMeans(n_clusters=k, n_init='auto').fit(X)
    labels = kmeans.labels_
    siluetas.append(silhouette_score(X, labels, metric='euclidean'))

plt.figure(figsize=(7,5))
plt.plot(k_values,siluetas,color='red')
plt.axvline(x=3,linestyle='dashed',color='gray')
plt.xticks(k_values)
plt.ylabel("Silhoutte Scores", fontsize=15)
plt.xlabel("Value of k", fontsize=15)
plt.show()

**Adjusted Mutual Information (AMI)**

Es un score que asigna una puntuación a la comparación entre dos clusterings. La Información Mutua Ajustada mide que tanta información comparten dos clusterings en términos de los elementos que comparten, es decir, del tamaño de la intersección.

Suele usarse para comparar un clustering *ground truth* contra uno que hemos obtenido. **Es forzoso tener dos clusterings.**

Usaremos la implementación de [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_mutual_info_score.html).




In [ ]:
from sklearn.metrics import adjusted_mutual_info_score

max_num_clusters = 20

scores = []
k_values = list(range(2,max_num_clusters))
for k in k_values:
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(X)
    labels = kmeans.labels_
    scores.append(adjusted_mutual_info_score(y, labels))

plt.figure(figsize=(7,5))
plt.plot(k_values,scores,color='red')
plt.axvline(x=3,linestyle='dashed',color='gray')
plt.xticks(k_values)
plt.ylabel("Adjusted Mutual Information", fontsize=15)
plt.xlabel("Value of k", fontsize=15)
plt.show()

Como podemos ver, en los tres casos se valida la hipótesis de que el mejor valor para $K$ es $K=3$. Esta hipótesis tiene mayor peso por el conocimiento previo del problema, es decir, al generar los datos sabiamos que teniamos 3 grupos de puntos.

### K-Means es sensible a outliers

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=500,centers=3, random_state=174)

plt.figure()
plt.scatter(X[:,0],X[:,1])
plt.show()

In [ ]:
Xp = X.copy()

Xp[2] = Xp[2] + [4,0]
Xp[5] = Xp[5] + [4.25,-1]
Xp[1] = Xp[1] + [-3,0]

plt.figure()
plt.scatter(Xp[:,0],Xp[:,1],c=y)
plt.scatter(Xp[[1,2,5],0],Xp[[1,2,5],1],marker='x',color='black')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
labels = kmeans.labels_

kmeans_p = KMeans(n_clusters=3)
kmeans_p.fit(Xp)
labels_p = kmeans_p.labels_

fig, axs = plt.subplots(1,2,figsize=(9,5),sharey=True)
axs[0].scatter(X[:,0],X[:,1],c=labels)
axs[1].scatter(Xp[:,0],Xp[:,1],c=labels_p)
fig.show()

In [ ]:
from sklearn.metrics import silhouette_score

max_num_clusters = 20

siluetas_1 = []
siluetas_2 = []
k_values = list(range(2,max_num_clusters))
for k in k_values:
    kmeans_1 = KMeans(n_clusters=k, n_init='auto').fit(X)
    labels_1 = kmeans_1.labels_
    siluetas_1.append(silhouette_score(X, labels_1, metric='euclidean'))
    kmeans_2 = KMeans(n_clusters=k, n_init='auto').fit(X)
    labels_2 = kmeans_2.labels_
    siluetas_2.append(silhouette_score(Xp, labels_2, metric='euclidean'))

fig, axs = plt.subplots(1,2)
fig.suptitle("Silhoutte Scores")
axs[0].plot(k_values,siluetas_1,color='red')
axs[0].set_title("Original dataset")
axs[1].plot(k_values,siluetas_2,color='red')
axs[1].set_title("Con outliers")
plt.show()

## [Clustering Jerárquico](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

* La agrupación jerárquica es una familia general de algoritmos de agrupación que crean agrupaciones anidadas fusionándolas o dividiéndolas sucesivamente. Esta jerarquía de grupos se representa como un árbol (o dendrograma). La raíz del árbol es el grupo único que reúne todas las muestras, siendo las hojas los grupos con una sola muestra.
* La implementación [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering) de scikit-learn realiza una agrupación jerárquica utilizando un enfoque ascendente (bottom-up): cada observación comienza en su propio grupo, y los grupos se fusionan sucesivamente. Los criterios de vinculación (_linkage_) determinan la métrica utilizada para la estrategia de fusión:
  - _Ward_ minimiza la suma de las diferencias al cuadrado dentro de todos los grupos. Es un enfoque que minimiza la varianza y, en este sentido, es similar a la función objetivo de k-means pero se aborda con un enfoque jerárquico aglomerativo.
  - _Maximum_ o _complete Linkage_ minimiza la distancia máxima entre observaciones de pares de grupos.
  - _Average linkage_ minimiza el promedio de las distancias entre todas las observaciones de pares de grupos.
  - _Single linkage_ minimiza la distancia entre las observaciones más cercanas de pares de grupos.



---


Se puede especificar, ya sea el número de clusters o el umbral de distancia máxima:

* Si `n_clusters`$\geq2$ entonces nos regresa ese número de clusters.
* Si `n_clusters`=None, hay que especificar un `distance_threshold`.
* Si `distance_threshold`$\neq$None, `n_clusters` debe ser None y `compute_full_tree` debe ser True.


In [ ]:
from sklearn.cluster import AgglomerativeClustering

# modelo = AgglomerativeClustering(n_clusters=3)
# no_dendo = True

# descomenta la siguiente línea si quieres ver un dendograma
modelo = AgglomerativeClustering(distance_threshold=5.0,
                                 n_clusters=None,
                                 compute_full_tree=True)
no_dendo=False


yhat = modelo.fit_predict(X)

clusters = np.unique(yhat)

for cluster in clusters:
    fila = np.where(yhat == cluster)
    plt.scatter(X[fila, 0], X[fila, 1])
plt.show()

print(f"{len(clusters)} clusters encontrados.")

También podemos usar criterios externos para escoger un número de clusters adecuado. Por ejemplo, podemos usar el score de silueta. La conclusión es usar 3 clusters.

In [ ]:
from sklearn.metrics import silhouette_score

max_num_clusters = 20

siluetas = []
k_values = list(range(2,max_num_clusters))
for k in k_values:
    ac = AgglomerativeClustering(n_clusters=k)
    ac.fit(X)
    labels = ac.labels_
    siluetas.append(silhouette_score(X, labels, metric='euclidean'))

plt.figure(figsize=(7,5))
plt.plot(k_values,siluetas,color='red')
plt.axvline(x=3,linestyle='dashed',color='gray')
plt.xticks(k_values)
plt.ylabel("Silhoutte Scores", fontsize=15)
plt.xlabel("Value of k", fontsize=15)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

if not no_dendo:
    plt.figure(dpi=120)
    plt.title('Hierarchical Clustering Dendrogram')
    # plot the top three levels of the dendrogram
    plot_dendrogram(modelo, truncate_mode='level', p=3)
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    plt.show()

## [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)
* [Martin Ester, Hans-Peter Kriegel, Jörg Sander, Xiaowei Xu (1996). _A Density-Based Algorithm for Discovering Clusters in Large Spatial Databases with Noise_. Proceedings of Knowledge Discovery and Databases - The International Conference on Knowledge Discovery & Data Mining.](https://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf)
* DBSCAN recurre a una noción de cúmulos basada en la densidad de los mismos, que está diseñada para descubrir grupos de formas arbitrarias. DBSCAN requiere solo un parámetro de entrada $\varepsilon$, el cual determina la distancia máxima entre dos puntos para considerarse cercanos. El otro parámetro importante es el `min_samples` el cual representa el número mínimo de puntos que puede haber en un cluster.

* DBSCAN reconoce ruido. Es decir, puede dejar puntos sin asignar a ningún cluster.

* "_La razón principal por la que reconocemos los grupos, es que dentro de cada grupo tenemos una densidad típica de puntos que es considerablemente más alta que fuera del grupo. Además, la densidad dentro de las áreas de ruido es menor que la densidad en cualquiera de los grupos._"

In [ ]:
from sklearn.cluster import DBSCAN

modelo = DBSCAN(eps=0.530, min_samples=9)

yhat = modelo.fit_predict(X)

clusters = [j for j in np.unique(yhat) if j!=-1]

plt.figure()
plt.scatter(X[yhat==-1,0],X[yhat==-1,1],marker='x',color='gray',label='ruido')
for cluster in clusters:
    filas = np.where(yhat == cluster)
    plt.scatter(X[filas, 0], X[filas, 1])
plt.legend(loc='best')
plt.show()

Una particularidad de DBSCAN es la susceptibilidad de los resultados en función del principal parámetro $\varepsilon$.

In [ ]:
valores_eps = np.linspace(0.0001,10,100)
num_clusters = []

for eps in valores_eps:
    modelo = DBSCAN(eps=eps, min_samples=5)
    yhat = modelo.fit_predict(X)
    clusters = np.unique(yhat)
    num_clusters.append(len(clusters))

plt.plot(valores_eps,num_clusters)
plt.suptitle("Número de clusters en función del parámetro eps")
plt.xlabel("eps",fontsize=15)
plt.ylabel("Número de clusters", fontsize=15)
plt.show()

## Otros métodos

### [Affinity Propagation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html)

* [Brendan J. Frey, Delbert Dueck (2007). Clustering by passing messages between data points. Science 315 (5814); pp. 972-6.](https://pdfs.semanticscholar.org/ea78/2c8b0848987e9575ea648e0419054d3f5bbf.pdf?_ga=2.62870572.1030401696.1591021245-1055786045.1581021538)

* AffinityPropagation crea clusters enviando mensajes entre pares de muestras hasta la convergencia. Los mensajes enviados entre pares representan la idoneidad de una muestra para ser el ejemplar de la otra, que se actualiza en respuesta a los valores de otros pares. Esta actualización se produce de forma iterativa hasta la convergencia, momento en el que se eligen los ejemplares definitivos y, por tanto, se obtiene la agrupación final.

* AffinityPropagation puede ser interesante, ya que elige el número de clusters en función de los datos proporcionados. Para ello, los dos parámetros importantes son la preferencia, que controla cuántos ejemplares se utilizan, y el factor de amortiguación, que amortigua los mensajes de responsabilidad y disponibilidad para evitar oscilaciones numéricas al actualizar estos mensajes.

* El principal inconveniente de la propagación por afinidad es su complejidad.

In [ ]:
from sklearn.cluster import AffinityPropagation

modelo = AffinityPropagation(damping=0.5,max_iter=500,random_state=None)

modelo.fit(X)
yhat = modelo.fit_predict(X)

clusters = np.unique(yhat)

print(f"Se encontraron {clusters.shape[0]} clusters")

for cluster in clusters:
    fila = np.where(yhat == cluster)
    plt.scatter(X[fila, 0], X[fila, 1])

plt.show()

### [BIRCH](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html)
* Tian Zhang, Raghu Ramakrishnan and Miron Livny (1996). _BIRCH: An Efficient Data Clustering Method for Very Large Databases_.  ACM SIGMOD Record. DOI:10.1145/235968.233324.
* BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies) es un método de clustering de tipo jerárquico bottom-up, especialmente diseñado para grandes bases de datos.
* BIRCH agrupa de forma incremental y dinámica datos métricos (que satisfacen los requisitos de una métrica Euclidiana) multidimensionales de entrada para intentar producir la mejor calidad de agrupación con los recursos disponibles (es decir, memoria disponible y limitaciones de tiempo).
* BIRCH es también el primer algoritmo de agrupamiento propuesto en el área de base de datos para manejar ruido (puntos que no son parte del patrón subyacente) de manera efectiva.
* El algoritmo BIRCH se basa en la construcción iterativa de un árbol cuyos nodos son tripletas (llamadas Clustering Features -- $CF$) que resumen la información acerca de un cluster; $CF=(N,LS,SS)$, donde $N$ es el número de puntos en un cluster, LS es la suma lineal de los N puntos ($LS=\sum_i X_i$) y $SS$ es la suma al cuadrado de los N puntos ($SS=\sum_i X_i^2$). La idea de CF es mantener una representación compacta de los clusters ya que su aglomeración se calcula fácilmente como $CF_i+CF_j=(N_i+N_j,LS_i+LS_j,SS_i+SS_j)$.
* La idea es construir el árbol de manera iterativa con base en un factor de ramificación B y un umbral T. Se parte de un nodo raíz al cuál se van agregando hijos ($child$'s) con base en una heurística.
* Cada nodo no-hoja contiene cuando mucho B entradas, y un nodo hoja representa un cluster hecho de todos los subclusters representados por sus entradas de forma $[CF_i]$, donde $i=1,2,\ldots,L$. Pero todas las entradas de un nodo hoja deben respetar el valor de umbral T, que representa la cota superior del diámetro (o radio) del cluster. El tamaño del árbol es función de T. Cuanto más grande, más pequeño el árbol.

In [ ]:
from sklearn.cluster import Birch

modelo = Birch(threshold=0.01, n_clusters=3)

modelo.fit(X)
yhat = modelo.predict(X)

clusters = np.unique(yhat)

for cluster in clusters:
    fila = np.where(yhat == cluster)
    plt.scatter(X[fila, 0], X[fila, 1])

plt.show()

### [OPTICS](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html)

OPTICS (Ordering Points To Identify the Clustering Structure) está muy relacionado con DBSCAN, también encuentra puntos *nucleo* y expande a partir de ellos. A diferencia de DBSCAN, mantiene una jerarquía de clusters para un intervalo de radios pequeños. Es una mejor alternativa a DBSCAN en datasets grandes.

In [ ]:
from sklearn.cluster import OPTICS

modelo = OPTICS(max_eps=200, min_samples=9)

yhat = modelo.fit_predict(X)

clusters = np.unique(yhat)

for cluster in clusters:
    filas = np.where(yhat == cluster)
    plt.scatter(X[filas, 0], X[filas, 1])

plt.show()

In [ ]:
num_clusters = []
valores_eps = np.linspace(0.0001,200,100)

for eps in valores_eps:
    modelo = OPTICS(eps=eps, min_samples=5)
    yhat = modelo.fit_predict(X)
    clusters = np.unique(yhat)
    num_clusters.append(len(clusters))

plt.plot(valores_eps,num_clusters)
plt.suptitle("Número de clusters en función del parámetro eps")
plt.xlabel("eps",fontsize=15)
plt.ylabel("Número de clusters", fontsize=15)
plt.show()

### [Spectral Clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html)

* La agrupación espectral es una clase general de métodos de agrupación, extraída del álgebra lineal.

* "_Una alternativa prometedora que ha surgido recientemente en varios campos es utilizar métodos espectrales para la agrupación. Aquí, uno usa los vectores propios más altos de una matriz derivada de la distancia entre puntos._" [Andrew Y. Ng, Michael I. Jordan and Yair Weiss (2002). _On Spectral Clustering: Analysis and an algorithm_. In ADVANCES IN NEURAL INFORMATION PROCESSING SYSTEMS.](https://papers.nips.cc/paper/2092-on-spectral-clustering-analysis-and-an-algorithm.pdf)

* Un review del método: https://link.springer.com/article/10.1007/s11222-007-9033-z

* El hiperparámetro "n_clusters" es utilizado para especificar el número estimado de clústeres en los datos.

In [ ]:
from sklearn.cluster import SpectralClustering

modelo = SpectralClustering(n_clusters=2)

yhat = modelo.fit_predict(X)

clusters = np.unique(yhat)

for cluster in clusters:
    fila = np.where(yhat == cluster)
    plt.scatter(X[fila, 0], X[fila, 1])

plt.show()

## Más métricas

Hay más métricas que pueden usarse para el clustering. Las hay de dos tipos:

1. Cuando no se conoce un clustering *ground truth*, en este caso se evalua el módelo unicamente con la información de él mismo. Por ejemplo:

    * [Silhoutte score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score)
    * [Calinski-Harabasz Index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score)
    * [Davies-Bouldin Index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score)
    * ...

2. Cuando se conoce un clustering *ground truth* o se quieren comparar dos clusterings. Por ejemplo:
    * [AMI](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_mutual_info_score.html#sklearn.metrics.adjusted_mutual_info_score)
    * [V-measure](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.v_measure_score.html#sklearn.metrics.v_measure_score)
    * [Rand index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.rand_score.html#sklearn.metrics.rand_score)
    * ...


## ⭕ Práctica

¿Puedes encontrar buenos clusterings para los siguientes datasets?

Considera dos datasets DS1 y DS2/DS3. En cada uno de ellos, prueba los siguientes métodos:

1. K-Means
2. AgglomerativeClustering
3. DBSCAN

Tareas a realizar:

* Realiza una busqueda de hiperparámetros basándote en alguna métrica como AMI, Silhoutte, Elbow Value (para el caso de K-Means) o alguna otra.
* Con estos hiperparámetros, escoge el mejor clustering de cada uno de los tres métodos.
* Compara visualmente los 3 clusterings obtenidos.
* Usando el score de silueta y el índice Calinski-Harabasz, ¿cuál de los tres clusterings fue mejor?
* En el caso del dataset DS1, tienes un *ground truth* clustering. Reporta el valor de la métrica [AMI](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_mutual_info_score.html#sklearn.metrics.adjusted_mutual_info_score) y [ARI](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html#sklearn.metrics.adjusted_rand_score).

In [ ]:
from sklearn.datasets import make_moons, make_blobs
import numpy as np

n_samples = 500

DS1 = make_moons(n_samples=n_samples, noise=.05)
DS2 = np.random.rand(n_samples, 2)
DS3 = make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=170)

In [ ]:
import matplotlib.pyplot as plt

# X, y = DS1
X, y = DS3
# X = DS2

plt.figure()
plt.scatter(X[:,0],X[:,1])
plt.show()

Graficar los clusters:

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(9,5),sharey=True)
axs[0].scatter(X[:,0],X[:,1], c=y)
axs[0].set_title("Original dataset")
axs[1].scatter(X[:,0],X[:,1], c=y_clusters)
axs[1].set_title("Clustering")
fig.show()

# Ejemplo 1: Documentos de Wikipedia

Usaremos otra vez el dataset de documentos de Wikipedia, tomaremos la versión parcial y preprocesada de la sesión pasada.

El objetivo de la práctica es segmentar los documentos en grupos con temáticas similares. Para esto, usaremos algoritmos de clustering. Si las representaciones vectoriales de los documentos son *buenas*, lograremos este objetivo.

In [ ]:
!pip install wordcloud -qq

In [ ]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/03%20Machine%20Learning/data/spanish-wikipedia-dataframe.csv"
df = pd.read_csv(url,index_col=0)
df

Nos quedamos con documentos que tengan entre 50 y 200 palabras

In [ ]:
df['Palabras'] = df['Texto'].apply(lambda x: x.split())
df['Total'] = df['Palabras'].apply(lambda x: len(x))
df.drop(columns=['Palabras'],inplace=True)

df = df[(df['Total'] < 200) & (df['Total'] > 50)]
df.reset_index(drop=True,inplace=True)
df

Construimos la matriz BOW de los documentos y hacemos PCA para obtener una matriz de caracteristicas (numéricas continuas).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

docs_list = df['Texto'].values

cv = CountVectorizer(max_features=None)
X_bow = cv.fit_transform(docs_list)
print(X_bow.shape)

pca = PCA(svd_solver='auto')
X_pca = pca.fit_transform(np.asarray(X_bow.todense()))
print(X_pca.shape)

Tomamos las primeras `n_dim` componentes principales como representación vectorial de cada documento.

In [ ]:
n_dim = 20

X_pca_dim = X_pca[:,:n_dim]
print(X_pca_dim.shape)

Hacemos clustering a las representaciones vectoriales.

In [ ]:
from sklearn.cluster import KMeans

clustering = KMeans(n_clusters=3)
clustering.fit(X_pca_dim)
clusters = clustering.labels_

In [ ]:
idxs_per_cluster = {j:np.where(clusters==j)[0] for j in np.unique(clusters)}
documents_per_cluster = {j:df.loc[idxs_per_cluster[j],'Texto'].values for j in np.unique(clusters)}

documents_per_cluster

Con la finalidad de explorar el contenido de los textos de cada cluster, hacemos una nube de palabras de los documentos de cada cluster. Para esto, usamos el módulo [wordcloud](https://pypi.org/project/wordcloud/).

Aquí puedes ver [ejemplos de su uso](https://github.com/amueller/word_cloud/tree/main)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(15,5),dpi=100)
for k,ax in enumerate(axs):
    wordcloud = WordCloud().generate(" ".join(documents_per_cluster[k]))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis("off")
    ax.set_title(f"Cluster {k}")
fig.show()

In [ ]:
from sklearn.metrics import silhouette_score

print(f"Score de silueta: {silhouette_score(X_pca_dim,clusters)}")

❓

* ¿Puedes ver de qué tratan los documentos de cada cluster?
* Prueba a cambiar el número de clusters.
* Prueba a cambiar el número de dimensiones de las representaciones vectoriales.
* Prueba a cambiar el método de clustering
* En cada tarea de clustering, mide el coeficiente de silueta.

# Ejemplo 1: Canciones de Spotify

Este conjunto de datos contiene estadísticas de audio de las 2.000 canciones top de Spotify. Los datos contienen alrededor de 15 columnas que describen la canción y algunas de sus cualidades. Se incluyen canciones publicadas desde 1956 hasta 2019 de algunos artistas notables y famosos. Estos datos contienen características de audio como Danceability, BPM, Liveness, Valence(Positivity) y algunas más:

* Índice: ID
* Título: Nombre de la pista
* Artista: Nombre del artista
* Género superior: Género de la pista
* Año: Año de lanzamiento de la pista
* Pulsaciones por minuto (BPM): El tempo de la canción
* Energy: La energía de una canción: cuanto más alto sea el valor, más energética será la canción.
* Danceability: Cuanto más alto sea el valor, más fácil será bailar esta canción.
* Loudness: Cuanto más alto sea el valor, más fuerte será la canción.
* Liveness: ...
* Valence: Cuanto más alto sea el valor, más positivo será el estado de ánimo de la canción.
* Duración: La duración de la canción.
* Acousticness: Cuanto más alto sea el valor, más acústica será la canción.
* Speechiness: Cuanto más alto sea el valor, más palabras habladas contiene la canción.
* Popularity: Cuanto más alto sea el valor, más popular es la canción.

Este dataset se encuentra en [Kaggle](https://www.kaggle.com/datasets/iamsumat/spotify-top-2000s-mega-dataset)

In [ ]:
import pandas as pd
import numpy as np

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/spotify-2000.csv'
df = pd.read_csv(url,index_col=0,thousands=',')
df

In [ ]:
# print(df['Length (Duration)'].values)
df.dtypes

In [ ]:
generos = df['Top Genre'].unique()
print(f"Hay {len(generos)} géneros únicos:")
print(generos)

In [ ]:
df.describe()

A manera de análisis exploratorio, veamos las correlaciones entre variables, ¿qué observamos?

In [ ]:
from seaborn import heatmap
import matplotlib.pyplot as plt

correlaciones = df.iloc[:,3:].corr()
heatmap(correlaciones)
plt.show()

Dado que algunos métodos de clustering son susceptibles a la escala de valores, hacemos un escalamiento.

In [ ]:
from sklearn.preprocessing import MinMaxScaler


df2 = df[["Beats Per Minute (BPM)", "Loudness (dB)",
              "Liveness", "Valence", "Acousticness",
              "Speechiness"]].copy()

scaler = MinMaxScaler()
df2[df2.columns] = scaler.fit_transform(df2[df2.columns])
X = df2.values

df2.head(3)

In [ ]:
df2.describe()

Usamos K-means para segmentar en 10 grupos

In [ ]:
from sklearn.cluster import KMeans

modelo = KMeans(n_clusters=10, n_init='auto')

modelo.fit(X)
clusters = modelo.labels_

print(f"Las primeras 10 etiquetas: {clusters[:10]}")

Integramos la información de los clusters al dataframe original.

In [ ]:
df["Music Segments"] = clusters
df["Music Segments"] = df["Music Segments"].map({0: "Cluster 1", 1:
    "Cluster 2", 2: "Cluster 3", 3: "Cluster 4", 4: "Cluster 5",
    5: "Cluster 6", 6: "Cluster 7", 7: "Cluster 8",
    8: "Cluster 9", 9: "Cluster 10"})
df.head(5)

Observemos un cluster

In [ ]:
cluster = 'Cluster 3'

df[df['Music Segments']==cluster][['Artist','Title','Top Genre','Year']]

Podemos ver los artistas en este cluster

In [ ]:
df[df['Music Segments']==cluster]['Artist'].unique()

Graficamos usando solamente 3 features. Usamos el módulo [plotly](https://plotly.com/python/) para gráficas interactivas.

Otra alternativa es [Bokeh](https://bokeh.org/).

In [ ]:
import plotly.graph_objects as go

PLOT = go.Figure()

for i in list(df["Music Segments"].unique()):
    PLOT.add_trace(go.Scatter3d(x = df[df["Music Segments"]==i]['Beats Per Minute (BPM)'],
                                    y = df[df["Music Segments"] ==i]['Energy'],
                                    z = df[df["Music Segments"] ==i]['Danceability'],
                                    mode = 'markers',marker_size = 6, marker_line_width = 1,
                                    name = str(i)))
PLOT.update_traces(hovertemplate='Beats Per Minute (BPM): %{x} <br>Energy: %{y} <br>Danceability: %{z}')

PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'Beats Per Minute (BPM)', titlefont_color = 'black'),
                                yaxis=dict(title = 'Energy', titlefont_color = 'black'),
                                zaxis=dict(title = 'Danceability', titlefont_color = 'black')),
                   font = dict(family = "Arial", color  = 'black', size = 12))

PLOT.show()

Usando sólo dos dimensiones:

In [ ]:
plt.figure(dpi=120)
for segment in df["Music Segments"].unique():
    plt.scatter(x = df[df["Music Segments"]==segment]['Beats Per Minute (BPM)'],
                y = df[df["Music Segments"] ==segment]['Energy'])
plt.show()

In [ ]:
plt.figure(dpi=120)
for segment in df["Music Segments"].unique():
    plt.scatter(x = df[df["Music Segments"]==segment]['Danceability'],
                y = df[df["Music Segments"] ==segment]['Energy'])
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

print(f"Score de silueta: {silhouette_score(X,clusters)}")

⭕ Preguntas:
* Siendo K-Means, ¿por qué se no se ve la separación perfecta?

⭕ Ejercicio 1. Continuando con este método de K-Means:
* ¿Qué valor de K es mejor? Puedes usar cualquiera de los 3 criteros de arriba, empezando por el *elbow value*.
* Una vez que hayas escogido un valor para $K$, reportar los valores de las métricas de clustering: score de Silueta, [Calinski-Harabasz Index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score) y [Davies-Bouldin Index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score).

⭕ Ejercicio 2:

* Repetir el experimento, ahora usando Agglomerative Clustering y DBSCAN.
* ¿Puedes elevar las métricas de clustering? Considera las métricas score de Silueta y [Davies-Bouldin Index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score).
* Explora visualmente algunas canciones de los clusters, ¿tiene sentido el agrupamiento?

# Ejemplo 2: `20newsgroups`

Retomamos el corpus de documentos `20newsgroups`. Realizaremos un clustering en las representaciones de los documentos para ver qué documentos se agrupan juntos.

In [ ]:
#@title Descargamos el dataset
from sklearn.datasets import fetch_20newsgroups
import numpy as np

data_full = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'),
                               categories=['soc.religion.christian','sci.space','rec.autos'])

In [ ]:
docs = data_full.data
print(f"Número de documentos: {len(docs)}")
topics = data_full.target
print(f"Número de tópicos: {np.unique(topics).shape[0]}")

In [ ]:
docs[:2]

## Representaciones *term-document* y *tf-idf*

Hemos visto que los documentos se pueden representar por medio de vectores *sparse* cuyas componentes son conteos de las apariciones de ciertas palabras.

Otra forma parecida de representar documentos es por medio de vectores cuyas componentes ahora representan índices *tf-idf* (term frequency - inverse document frequency). Estas tienen la ventaja de restar importancia a palabras que aparecen en muchos documentos.

Para un corpus $D$ de documentos, el índice $tfidf$ de un término $t$ en un documento $d$ se calcula como

$$tfidf(t,d,D)=tf(t,d)idf(t,D)$$

donde

$$tf(t,d)=\frac{f_{t,d}}{\sum_{s\in d}f_{s,d}}$$

$$idf(t,D)=\log\frac{N}{|\{d\in D : t\in d\}|}$$

$f_{t,d}$ es el número de ocurrencias de un término $t$ en un documento $d$, $N$ es el número de documentos en $D$.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


corpus = [
        'This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?',
        ]

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(tfidf_vectorizer.get_feature_names_out())
print(X_tfidf)
print(X_tfidf.todense()[:,:3])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X_counts = count_vectorizer.fit_transform(corpus)
print(count_vectorizer.get_feature_names_out())
print(X_counts)
print(X_counts.todense()[:,:3])

## Clustering

Traemos la función para limpiar texto de hace algunas sesiones

In [ ]:
url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/02-Machine-Learning/data/limpiador_texto.py"
!wget --no-cache --backups=1 {url}

In [ ]:
from nltk import download

download('stopwords')
download('punkt')

In [ ]:
from limpiador_texto import preprocesar_textos

clean_docs = preprocesar_textos(docs)

Realizamos el clustering con las dos representaciones y calculamos las métricas de rendimiento del clustering.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score, silhouette_score


vectorizer = TfidfVectorizer(stop_words='english',
                             max_features=200)
X_tfidf = vectorizer.fit_transform(docs)
# X_tfidf = vectorizer.fit_transform(clean_docs)
print(X_tfidf.shape)

clustering = KMeans(n_clusters=3, n_init='auto')
clustering.fit(X_tfidf)
clusters = clustering.labels_

print(f"AMI: {adjusted_mutual_info_score(topics,clusters)}")
print(f"AR: {adjusted_rand_score(topics,clusters)}")
print(f"Silhoutte del clustering obtenido: {silhouette_score(X_tfidf,clusters)}")
print(f"Silhoutte de los tópicos: {silhouette_score(X_tfidf,topics)}")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score, silhouette_score


vectorizer = CountVectorizer(stop_words='english',
                             max_features=200)
# X_counts = vectorizer.fit_transform(docs)
X_counts = vectorizer.fit_transform(clean_docs)
print(X_counts.shape)

clustering = KMeans(n_clusters=3, n_init='auto',random_state=57)
clustering.fit(np.asarray(X_counts.todense()))
clusters_counts = clustering.labels_

print(f"AMI: {adjusted_mutual_info_score(topics,clusters_counts)}")
print(f"AR: {adjusted_rand_score(topics,clusters_counts)}")
print(f"Silhoutte del clustering obtenido: {silhouette_score(np.asarray(X_counts.todense()),clusters_counts)}")
print(f"Silhoutte de los tópicos: {silhouette_score(np.asarray(X_counts.todense()),topics)}")

Podríamos explorar el comportamiento del parámetro `n_clusters` usando los criterios de intertia y silhoutte.

In [ ]:
import matplotlib.pyplot as plt

sils = []
inertias = []

for n in range(2,20):
    clustering = KMeans(n_clusters=n, n_init='auto')
    clustering.fit(np.asarray(X_counts.todense()))
    # sil = silhouette_score(np.asarray(X_counts.todense()),clustering.labels_)
    # sils.append(sil)
    inertias.append(clustering.inertia_)


plt.plot(list(range(2,20)),inertias)
plt.show()

## Exploración de los clusters

Podemos probar algunas técnicas adicionales para explorar los clusters

In [ ]:
!pip install -qq wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(15,10),dpi=100)
for k,ax in enumerate(axs):
    cluster_text = " ".join([clean_docs[j] for j,cluster in enumerate(clusters_counts) if cluster==k])
    wc = WordCloud().generate(cluster_text)
    ax.set_title(f"Cluster {k}")
    ax.imshow(wc, interpolation='bilinear')
    ax.set_xticks([])
    ax.set_yticks([])
fig.show()

Imprimimos algunos documentos en cada cluster

In [ ]:
np.random.seed(43)

for k in range(3):
    docs_in_cluster = [clean_docs[j] for j,cluster in enumerate(clusters_counts) if cluster==k]
    print(f"Cluster {k} {20*'-'}")
    print(f"Número de documentos en el cluster: {clusters_counts[clusters_counts==k].shape[0]}")
    some_docs = np.random.choice(docs_in_cluster,size=3)
    print(some_docs)

⭕ **Ejercicio**

Repite el experimento variando el parámetro `max_features` de los vectorizadores para ver si puedes subir las métricas de rendimiento y obtener una mejor separación de tópicos, esto hazlo *visualmente* (usando las nubes de palabras y algunos documentos).

Usa los dos vectorizadores
